In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics as stat
import numpy as np

In [8]:
readings = pd.read_csv("data\modified_data.csv")
readings.drop(columns=["Unnamed: 0"], inplace=True)

In [59]:
subset = readings[(readings["building_id"] == 12) & (readings["value_type_id"] == 1)]
subset.drop(columns=["building_id", "date", "time", "value_type_id"], inplace=True)
subset["seconds"] /= 86400
subset

C:\Users\ariqi\AppData\Local\Temp\ipykernel_17956\1800094793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset.drop(columns=["building_id", "date", "time", "value_type_id"], inplace=True)
C:\Users\ariqi\AppData\Local\Temp\ipykernel_17956\1800094793.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["seconds"] /= 86400


,device_id,seconds,month,dweek,holiday,value
26,83,0.003391,12,0,False,401.0
30,70,0.082917,4,6,True,394.0
58,93,0.677639,12,4,False,417.0
72,81,0.991701,10,1,False,491.0
116,85,0.476181,2,4,False,430.0
...,...,...,...,...,...,...
39004068,84,0.828044,6,2,False,512.0
39004116,84,0.903970,6,0,False,505.0
39004117,71,0.483264,6,4,False,404.0
39004142,74,0.453912,11,1,False,455.0


In [60]:
mask = np.random.rand(len(subset)) < 0.75
train = subset[mask]
test = subset[~mask]


In [61]:
device_ids = subset["device_id"].unique()
#print(device_ids)

norm_transform = dict()

for x in device_ids:
    vals = train[train["device_id"] == x]["value"].to_list()
    norm_transform[x] = [stat.mean(vals), stat.stdev(vals)]    

id_subsets = []

for id in norm_transform:
    id_subset = train[train["device_id"] == id]
    id_subset["value"] = (id_subset["value"] - norm_transform[id][0]) / norm_transform[id][1]
    id_subsets.append(id_subset)

transformed = pd.concat(id_subsets)
transformed

C:\Users\ariqi\AppData\Local\Temp\ipykernel_17956\1098858310.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_subset["value"] = (id_subset["value"] - norm_transform[id][0]) / norm_transform[id][1]


,device_id,seconds,month,dweek,holiday,value
26,83,0.003391,12,0,False,-0.652472
3613,83,0.346157,2,5,True,-0.522432
5013,83,0.868125,2,3,False,-0.148567
7534,83,0.336528,5,1,False,-0.571197
8779,83,0.454838,1,6,True,-0.636217
...,...,...,...,...,...,...
38995524,67,0.333542,4,5,True,-0.187219
38996000,67,0.558426,10,6,True,-0.679281
38996660,67,0.776713,5,4,False,-0.740789
39001583,67,0.995463,6,2,False,-1.263606


In [62]:
x_train = transformed.drop(columns=["value"])
y_train = transformed["value"]



26         -0.652472
3613       -0.522432
5013       -0.148567
7534       -0.571197
8779       -0.636217
              ...   
38995524   -0.187219
38996000   -0.679281
38996660   -0.740789
39001583   -1.263606
39003746    0.704644
Name: value, Length: 422361, dtype: float64